In [37]:
!python /data/benny_cai/miniconda3/envs/brax/lib/python3.9/site-packages/brax/v1/tools/urdf_converter.py --xml_model_path 'legged_studio_a1_more_colli.urdf' --config_path 'a1'

I0125 20:41:11.544692 140288389826368 urdf_converter.py:58] Loading urdf model from legged_studio_a1_more_colli.urdf


In [1]:
# !python /data/benny_cai/miniconda3/envs/brax/lib/python3.9/site-packages/brax/v1/tools/mujoco_converter.py --xml_model_path 'a1_mjcf.txt' --config_path 'a1_mjcf'

In [41]:
from brax.generalized import pipeline
from brax.io import mjcf

from IPython.display import HTML, clear_output
import jax
from jax import numpy as jp
from brax.io import html
import json
import numpy as np

TF_CPP_MIN_LOG_LEVEL=0

In [18]:
from scipy.interpolate import CubicSpline

In [19]:
with open('a1_mjcf.txt', 'r') as file:
    config = file.read()

m = mjcf.loads(config, asset_path='')

In [20]:
file = 'a1_origin_motions/trot.txt'
with open(file, 'r') as f:
    data = json.load(f)
frames = data['Frames']
len(frames)

33

In [21]:
frames = jp.array(frames)
frames = frames[:, 7:]
frames[:5,0]

Array([-0.04085, -0.00562,  0.03923,  0.06805,  0.0373 ], dtype=float32)

In [22]:
frames = frames.T

In [23]:
time_points = jp.arange(frames.shape[1])
splines = [CubicSpline(time_points, frames[joint]) for joint in range(12)]
inter = 6
fine_time = jp.linspace(time_points.min(), time_points.max(), len(time_points)*inter)
interpolated_data = jp.array([spline(fine_time) for spline in splines])

In [24]:
frames = interpolated_data.T
frames.shape

(198, 12)

In [25]:
frames[:8,0]

Array([-0.04085   , -0.03612936, -0.03097054, -0.02540927, -0.01948133,
       -0.01322246, -0.00666841,  0.00014505], dtype=float32)

In [26]:
jit_env_reset = jax.jit(pipeline.init)
jit_env_step = jax.jit(pipeline.step)

rollout = []
rng = jax.random.PRNGKey(seed=1)
state = jit_env_reset(m, m.init_q, jp.zeros(m.qd_size()))
for frame in frames:
    rollout.append(state)
    act = jp.array(frame)
    # print(act)
    state = jit_env_step(m, state, act)

In [35]:
# HTML(html.render(m, rollout))

In [17]:
# dir(rollout[0])

In [18]:
# rollout[0].x

In [19]:
# rollout[0].xd

In [11]:
# state = jit_env_reset(m, m.init_q, jp.zeros(m.qd_size()))
# # HTML(html.render(m, [state]))

In [32]:
with open('a1_mjcf.txt', 'r') as file:
    config = file.read()

m = mjcf.loads(config, asset_path='')

In [66]:
file = 'a1_origin_motions/trot.txt'
with open(file, 'r') as f:
    data = json.load(f)
frames = data['Frames']
frames = np.array(frames)
frames.shape

(33, 19)

In [67]:
jit_env_reset = jax.jit(pipeline.init)
jit_env_step = jax.jit(pipeline.step)

rng = jax.random.PRNGKey(seed=1)
# state = jit_env_reset(m, m.init_q, jp.zeros(m.qd_size()))
rollout = []
for frame in frames:
    frame[3], frame[6] = frame[6], frame[3]
    state = jit_env_reset(m, frame, jp.zeros(m.qd_size()))
    rollout.append(state)

In [61]:
HTML(html.render(m, rollout))

In [68]:
data = []
for roll in rollout:
    dict = {'pos': roll.x.pos, 'rot': roll.x.rot, 'vel': roll.xd.vel, 'ang': roll.xd.ang}
    data.append(dict)

In [69]:
import pickle

file = 'a1_ref_motion/trot.pkl'
with open(file, 'wb') as pickle_file:
    pickle.dump(data, pickle_file)

In [70]:
len(data)

33

### Try

In [1]:
from brax.generalized import pipeline
from brax.io import mjcf

from IPython.display import HTML, clear_output
import jax
from jax import numpy as jp
from brax.io import html

In [2]:
with open('a1_mjcf.txt', 'r') as file:
    config = file.read()

m = mjcf.loads(config, asset_path='')

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [4]:
jit_env_reset = jax.jit(pipeline.init)
jit_env_step = jax.jit(pipeline.step)

rollout = []
rng = jax.random.PRNGKey(seed=1)
state = jit_env_reset(m, m.init_q, jp.zeros(m.qd_size()))
for i in range(10):
    print(i)
    rollout.append(state)
    # act = 10 * jp.sin(i / 100) * jp.ones(m.act_size())
    act = jp.zeros(m.act_size())
    state = jit_env_step(m, state, act)

0
1
2
3
4
5
6
7
8
9


In [25]:
# HTML(html.render(m, rollout))

In [6]:
m.init_q

Array([0.  , 0.  , 0.43, 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ], dtype=float32)

In [3]:
dir(m.actuator)

['T',
 '__add__',
 '__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_flax_dataclass',
 'bias_q',
 'bias_qd',
 'concatenate',
 'ctrl_range',
 'force_range',
 'gain',
 'gear',
 'index_set',
 'index_sum',
 'q_id',
 'qd_id',
 'replace',
 'reshape',
 'select',
 'slice',
 'take',
 'tree_replace',
 'vmap']

In [29]:
dir(m)

['T',
 '__add__',
 '__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_flax_dataclass',
 '_model',
 'act_size',
 'actuator',
 'ang_damping',
 'baumgarte_erp',
 'collide_scale',
 'concatenate',
 'density',
 'dof',
 'dof_link',
 'dof_ranges',
 'dt',
 'enable_fluid',
 'geom_masks',
 'geoms',
 'get_mjx_model',
 'get_model',
 'gravity',
 'index_set',
 'index_sum',
 'init_q',
 'joint_scale_ang',
 'joint_scale_pos',
 'link',
 'link_names',
 'link_parents',
 'link_types',
 'matrix_inv_iterations',
 'num_links',
 'q_idx',
 'q_size',
 'qd_idx',
 'qd_size',
 'rep

In [57]:
dir(m.link)

['T',
 '__add__',
 '__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_flax_dataclass',
 'concatenate',
 'constraint_ang_damping',
 'constraint_limit_stiffness',
 'constraint_stiffness',
 'constraint_vel_damping',
 'index_set',
 'index_sum',
 'inertia',
 'invweight',
 'joint',
 'replace',
 'reshape',
 'select',
 'slice',
 'take',
 'transform',
 'tree_replace',
 'vmap']

In [5]:
m.link.inertia

Inertia(transform=Transform(pos=Array([[ 0.00000e+00,  4.10000e-03, -5.00000e-04],
       [-3.31100e-03, -6.35000e-04,  3.10000e-05],
       [-3.23700e-03,  2.23270e-02, -2.73260e-02],
       [ 4.72659e-03,  0.00000e+00, -1.31975e-01],
       [-3.31100e-03,  6.35000e-04,  3.10000e-05],
       [-3.23700e-03, -2.23270e-02, -2.73260e-02],
       [ 4.72659e-03,  0.00000e+00, -1.31975e-01],
       [ 3.31100e-03, -6.35000e-04,  3.10000e-05],
       [-3.23700e-03,  2.23270e-02, -2.73260e-02],
       [ 4.72659e-03,  0.00000e+00, -1.31975e-01],
       [ 3.31100e-03,  6.35000e-04,  3.10000e-05],
       [-3.23700e-03, -2.23270e-02, -2.73260e-02],
       [ 4.72659e-03,  0.00000e+00, -1.31975e-01]], dtype=float32), rot=Array([[-1.8256460e-03,  7.0638072e-01, -6.4208568e-04,  7.0782948e-01],
       [ 5.0752789e-01,  5.0626791e-01,  4.9150690e-01,  4.9449891e-01],
       [ 9.9912524e-01, -2.5639306e-03, -4.0953111e-02, -8.0609117e-03],
       [ 7.0688641e-01,  1.7653009e-02,  1.7653009e-02,  7.068864

In [61]:
m.link.constraint_stiffness

Array([2000., 2000., 2000., 2000., 2000., 2000., 2000., 2000., 2000.,
       2000., 2000., 2000., 2000.], dtype=float32)

In [60]:
m.link.constraint_limit_stiffness

Array([1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000., 1000.,
       1000., 1000., 1000., 1000.], dtype=float32)

In [31]:
m.actuator.gain

Array([100., 100., 100., 100., 100., 100., 100., 100., 100., 100., 100.,
       100.], dtype=float32)

In [32]:
m.gravity

Array([ 0.  ,  0.  , -9.81], dtype=float32)

In [39]:
m.ang_damping

Array(0., dtype=float32)

In [40]:
m.baumgarte_erp

Array(0.1, dtype=float32)

In [42]:
m.vel_damping

Array(0., dtype=float32)

In [41]:
m.joint_scale_ang

Array(0.2, dtype=float32)

In [63]:
m.link.inertia

Inertia(transform=Transform(pos=Array([[ 0.00000e+00,  4.10000e-03, -5.00000e-04],
       [-3.31100e-03, -6.35000e-04,  3.10000e-05],
       [-3.23700e-03,  2.23270e-02, -2.73260e-02],
       [ 4.72659e-03,  0.00000e+00, -1.31975e-01],
       [-3.31100e-03,  6.35000e-04,  3.10000e-05],
       [-3.23700e-03, -2.23270e-02, -2.73260e-02],
       [ 4.72659e-03,  0.00000e+00, -1.31975e-01],
       [ 3.31100e-03, -6.35000e-04,  3.10000e-05],
       [-3.23700e-03,  2.23270e-02, -2.73260e-02],
       [ 4.72659e-03,  0.00000e+00, -1.31975e-01],
       [ 3.31100e-03,  6.35000e-04,  3.10000e-05],
       [-3.23700e-03, -2.23270e-02, -2.73260e-02],
       [ 4.72659e-03,  0.00000e+00, -1.31975e-01]], dtype=float32), rot=Array([[-1.8256460e-03,  7.0638072e-01, -6.4208568e-04,  7.0782948e-01],
       [ 5.0752789e-01,  5.0626791e-01,  4.9150690e-01,  4.9449891e-01],
       [ 9.9912524e-01, -2.5639306e-03, -4.0953111e-02, -8.0609117e-03],
       [ 7.0688641e-01,  1.7653009e-02,  1.7653009e-02,  7.068864

In [45]:
dir(m.get_model())

['__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_address',
 'actuator',
 'actuator_acc0',
 'actuator_actadr',
 'actuator_actearly',
 'actuator_actlimited',
 'actuator_actnum',
 'actuator_actrange',
 'actuator_biasprm',
 'actuator_biastype',
 'actuator_cranklength',
 'actuator_ctrllimited',
 'actuator_ctrlrange',
 'actuator_dynprm',
 'actuator_dyntype',
 'actuator_forcelimited',
 'actuator_forcerange',
 'actuator_gainprm',
 'actuator_gaintype',
 'actuator_gear',
 'actuator_group',
 'actuator_length0',
 'actuator_lengthrange',
 'actuator_plugin',
 'actuator_trnid',
 'actuator_trntype',
 'actuator_user',
 'body',
 'body_bvhadr',
 'body_

In [48]:
m.get_model().actuator_gaintype

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [40]:
from IPython.display import HTML, clear_output
import jax
from jax import numpy as jp

import brax
from brax.v1.envs import env
from brax.v1.io import json
from brax.v1.io import html

In [41]:
with open('a1', 'r') as file:
    content = file.read()

In [42]:
class A1(env.Env):
    def __init__(self, system_config):
        super().__init__(config=system_config)

    def reset(self, rng: jp.ndarray) -> env.State:
        qp = self.sys.default_qp()
        pos = qp.pos
        info = self.sys.info(qp)
        obs = jp.zeros(1)
        reward, done, zero = jp.zeros(3)
        metrics = {
            'torsoIsUp': zero,
            'torsoHeight': zero
        }
        info = {'rng': rng}
        return env.State(qp, obs, reward, done, metrics, info)

    def step(self, state: env.State, action: jp.ndarray) -> env.State:
        qp, info = self.sys.step(state.qp, action)
        return state.replace(qp=qp)

In [43]:
a1 = A1(content)

/data/benny_cai/miniconda3/envs/brax/lib/python3.9/site-packages/brax/v1/physics/base.py:193: UserWarning: dynamics_mode not specified, but joint.stiffness >0. Setting dynamics_mode="legacy_spring".
  warnings.warn('dynamics_mode not specified, but joint.stiffness >0. '


In [14]:
# env_state = a1.reset(rng=jax.random.PRNGKey(0))
# HTML(html.render(a1.sys, [a1.sys.default_qp()]))

In [58]:
env_state = a1.reset(rng=jax.random.PRNGKey(0))

# rollout = []
# for i in range(2):  # You can adjust the number of simulation steps
#     print(i)
#     # # Take a random action
#     # action = jax.random.uniform(jax.random.PRNGKey(0), shape=(a1.action_size,))
#     # action = jp.insert(action, 0,0)
#     # action = jp.append(action, 0)
#     action = jp.ones(a1.action_size,) * (-0.2)
    
#     # Step the environment
#     env_state = a1.step(env_state, action)

#     rollout.append(env_state.qp)

# rollout = []
# action = jp.ones(a1.action_size,) * (-0.01) * 0
# env_state = a1.step(env_state, action)
# rollout.append(env_state.qp)

# action = jp.ones(a1.action_size,) * (-0.01)
# env_state = a1.step(env_state, action)
# rollout.append(env_state.qp)

In [6]:
# HTML(html.render(a1.sys, rollout))

In [7]:
a1.action_size

12

In [10]:
a1.sys.num_joints

12

In [8]:
a1.sys.body.index

{'trunk': 0,
 'FR_hip': 1,
 'FR_upper': 2,
 'FR_lower': 3,
 'FL_hip': 4,
 'FL_upper': 5,
 'FL_lower': 6,
 'RR_hip': 7,
 'RR_upper': 8,
 'RR_lower': 9,
 'RL_hip': 10,
 'RL_upper': 11,
 'RL_lower': 12,
 'floor': 13}

In [11]:
a1.sys.body.inertia

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [9]:
a1.sys.default_qp()

QP(pos=array([[ 0.        ,  0.        ,  0.41992316],
       [ 0.183     , -0.047     ,  0.41992316],
       [ 0.183     , -0.13205   ,  0.41992316],
       [ 0.17751757, -0.13205   ,  0.21999831],
       [ 0.183     ,  0.047     ,  0.41992316],
       [ 0.183     ,  0.13205   ,  0.41992316],
       [ 0.17751757,  0.13205   ,  0.21999831],
       [-0.183     , -0.047     ,  0.41992316],
       [-0.183     , -0.13205   ,  0.41992316],
       [-0.18848243, -0.13205   ,  0.21999831],
       [-0.183     ,  0.047     ,  0.41992316],
       [-0.183     ,  0.13205   ,  0.41992316],
       [-0.18848243,  0.13205   ,  0.21999831],
       [ 0.        ,  0.        ,  0.        ]]), rot=array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 9.99906050e-01,  3.04364422e-18,  1.37073550e-02,
         0.00000000e+00],
       [ 9.99997886e-01, -4.56560494e-19, -2.05616567e-03,
 